In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import pandas as pd

In [2]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [3]:
df = pd.read_csv("train_dataset_train_variant2.csv")
bad_themes = ["Государственная собственность", "Культура", 'МФЦ "Мои документы"', "Памятники и объекты культурного наследия"
             "Погребение и похоронное дело", "Роспотребнадзор", "Спецпроекты", "Строительство и архитектура", "Торговля",
              "Физическая культура и спорт", "Экология", "Экономика и бизнес", "Электроснабжение"]

In [4]:
prompt = "[ИНФОРМАЦИЯ]:\nПользовательское обращение: {}\nИсполнитель, решающий проблему: {}\nГруппа темы: {}\nТема: {}\n[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] сгенирируй 5 пользовательских обращений схожих по смыслу соответствующих темам и верни их в python листе:"

In [5]:
from tqdm import tqdm

In [6]:
results = []

In [7]:
for theme in tqdm(bad_themes):
    for pod_theme in df[df["Группа тем"] == theme].Тема.unique().tolist():
        for i, dataframe in df[(df["Группа тем"] == theme) & (df["Тема"] ==pod_theme)].reset_index(drop=True).iterrows():
            
            if i == 2:
                break
            
            prompt_ = prompt.format(dataframe["Текст инцидента"], dataframe["Исполнитель"],
                                    theme, pod_theme)

            print(prompt_)
            conversation = Conversation()
            conversation.add_user_message(prompt_)
            prompt_ = conversation.get_prompt(tokenizer)
        
            output = generate(model, tokenizer, prompt_, generation_config)

            print(output)
            print('-------------------\n')
            result = [dataframe["Исполнитель"], theme, output, pod_theme]
            
            results.append(result)

            result_ = str([dataframe["Исполнитель"], theme, output, pod_theme])
            
            with open("result.txt", 'a') as file:
                file.write(result_+ '\n')

            torch.cuda.empty_cache()

  0%|          | 0/12 [00:00<?, ?it/s]

[ИНФОРМАЦИЯ]:
Пользовательское обращение:  Здравствуйте. Кто собственник бывшего училища 6 на ул.Революции? Средь бела дня ломают здание, выбивают стёкла, выносят металл. Или здание демонтируется собственником?
Исполнитель, решающий проблему: Лысьвенский городской округ
Группа темы: Государственная собственность
Тема: Региональное имущество
[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] сгенирируй 5 пользовательских обращений схожих по смыслу соответствующих темам и верни их в python листе:
1. Здравствуйте. Мне интересно, как можно получить доступ к государственной собственности в России?
2. Здравствуйте. Как определить, является ли объект государственной или муниципальной собственностью?
3. Здравствуйте. Какие виды государственной собственности существуют в России?
4. Здравствуйте. Как можно купить землю, которая находится в государственной собственности?
5. Здравствуйте. Как можно получить разрешение на использование объекта, который находится в государственной собственности?
------------------

  8%|▊         | 1/12 [01:31<16:44, 91.27s/it]

Здравствуйте, подскажите пожалуйста, куда можно обратиться по вопросам аренды свободного земельного участка?

Здравствуйте, как можно получить информацию о доступных для аренды земельных участках в городе?

Здравствуйте, есть ли возможность аренды земельного участка для строительства частного дома?

Здравствуйте, как можно узнать стоимость аренды земельного участка в городе?

Здравствуйте, какие документы необходимы для аренды земельного участка?
-------------------

[ИНФОРМАЦИЯ]:
Пользовательское обращение:   задал вопрос про статус модерации кинотеатра в Александровске. Мне не ответили. Почему?
Исполнитель, решающий проблему: Александровский муниципальный округ Пермского края
Группа темы: Культура
Тема: Учреждения культуры
[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] сгенирируй 5 пользовательских обращений схожих по смыслу соответствующих темам и верни их в python листе:
1. Задал вопрос о статусе модерации кинотеатра в Александровске. Меня не ответили. Почему?
2. Хотел узнать, есть ли возможн

 17%|█▋        | 2/12 [05:31<29:50, 179.01s/it]

...

Пермь г, 79918071863. Здравствуйте. Я обращаюсь с просьбой о помощи в благоустройстве Утиного болота. В этом месте обитают множество птиц, включая красавцев - кряквы. Однако, из-за неадекватной ситуации с мусором, которые оставляют туристы, место стало непривлекательным для этих животных. Также, число уток здесь только уменьшилось. Пожалуйста, постарайтесь найти решение данной проблемы.

Пермь г, 79918081848. Здравствуйте. Я обращаюсь с вопросом о благоустройстве Утиного болота. В 2021 году я уже писала о том, что хотят сделать, но не знаю, что было сделано. Однако, указатели были убраны. Но, кажется, болото не имеет хозяина или куратора, который мог бы заботиться о его благоустройстве. В результате, место стало непривлекательным для кряквы, а количество уток только уменьшилось. Пожалуйста, подумайте над решением данной проблемы.

Пермь г. 79918071848. Здаравствуйте. Я обращаюсь с просьбою о помощи в благоустройстве Утиного болова. В этом месте обитают множество птицы, включая красавце

 92%|█████████▏| 11/12 [42:02<04:16, 256.57s/it]

1. Привет! Я хочу узнать, как можно оптимизировать расходы на товары и услуги для своей компании?
2. Здравствуйте! Мне интересно, как вычислить среднюю цену на товар или услугу в определенной отрасли?
3. Привет! Я хотел бы узнать, как можно снизить затраты на материалы и сырье для производства?
4. Привет! Мне интересно, как можно снизить затраты на энергоресурсы для дома или офиса?
5. Привет! Мне интересно, как можно оптимизировать расходы на транспортные средства для своей компании?
-------------------

[ИНФОРМАЦИЯ]:
Пользовательское обращение:  Здравствуйте у нас в деревне зязелга не хватает напряжение даже воды набрать не можем
Исполнитель, решающий проблему: Бардымский муниципальный округ Пермского края
Группа темы: Электроснабжение
Тема: Качество электроснабжения
[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] сгенирируй 5 пользовательских обращений схожих по смыслу соответствующих темам и верни их в python листе:
1. Здравствуйте! У нас в деревне есть проблема с электроснабжением - напряжение

100%|██████████| 12/12 [45:40<00:00, 228.40s/it]

1. На улице Интернациональная с.Барда прошит кабель через проезжую часть, после ремонта инженерных сетей на опоре ЛЭП.
2. На улице Интернациональная с.Барда установлен кабель через проезжую часть, после ремонта электросети на опоре ЛЭП.
3. На улице Интернациональная с.Барда подвешен кабель через проезжую часть, после ремонта системы освещения на опоре ЛЭП.
4. На улице Интернациональная с.Барда перекинут кабель через проезжую часть, после ремонта коммуникационной сети на опоре ЛЭП.
5. На улице Интернациональная с.Барда выведен кабель через проезжую часть, после ремонта водопровода на опоре ЛЭП.
-------------------



In [ ]:
df = pd.read_csv("train_dataset_train_variant2.csv")
bad_themes = ["Государственная собственность", "Культура", 'МФЦ "Мои документы"', "Памятники и объекты культурного наследия"
             "Погребение и похоронное дело", "Роспотребнадзор", "Спецпроекты", "Строительство и архитектура", "Торговля",
              "Физическая культура и спорт", "Экология", "Экономика и бизнес", "Электроснабжение"]